# Notebook 4: Pandas and TF binding site analysis

Here we will start using Pandas. Pandas is the standard way of working with columnar data. However, there is a substantial learning curve. If you want to learn more about Pandas, here is a useful site: http://pandas.pydata.org/

Here we will use Pandas to analyze transcription factor (TF) binding sites from *Escherichia coli*. We will first focus on CRP, a major regulator in *E. coli* with over 350 functional binding sites.  

## Installation and downloads

In [ ]:
!pip -q install palmerpenguins
!pip -q install logomaker

## Introduction

Pandas is a library for working with tabular data. It was orignally based on the R data.frame library, but with a slightly different grammer and some different functionality. 

There are two main types of objects in Pandas.

1)`Dataframe`

In [ ]:
from palmerpenguins import load_penguins
df = load_penguins()
print(type(df))
df

* A 2-D object
* Has row (index) and column names
* The orientation of rows vs columns matters a lot
* Generally, you want features as columns and observations as rows
* Features are variables, they are the things you measure, whether that be quantitatively or qualitatively. While observations are each data point, in this case it is each penguin

2) `pd.Series`


In [ ]:
series = df['species']
print(type(series))
series

* A single column of data
* Contains rownames but no column name, the rownames are always reffered to as `pd.Series.index`
* Can have an attribute `pd.Series.name` that can serve as the column name
* Works a lot like a python dictionary


### Things we need to learn 
1) How to create a pandas object/ read data in

2) How to subset your data

3) How to manipulate/mutate your data to create more data

4) How to Summarize or aggregate your data



## Preparing Code 1

In [ ]:
# Put this first
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import logomaker; we will use this later for visualizing TF motifs.
import logomaker 

In [ ]:
# We will be analyzing a standing database of TF binding sites, which is available on RegulonDB. 
# Here is a command for downloading this file (this didn't work in lecture 1)
!mkdir data 
!wget -O ./data/binding_site_db.txt http://regulondb.ccg.unam.mx/menu/download/datasets/files/BindingSiteSet.txt 

In [ ]:
# Let's see what this database looks like
!head -n 50 data/binding_site_db.txt

## Reading data

When you store your data as a file you can use one of the `pd.read_*` functions to read in data from a variety of different file types

Then to save your progress you can write using `pd.DataFrame.write_*`


There are quite a lot of file types, and they all have their pros/cons. The simpliest and most commonly used one is `.csv`. 

In [ ]:
# To parse this file, use Pandas's method read_csv. 
df = pd.read_csv("data/binding_site_db.txt", sep='\t', comment='#', header=None)
type(df)

In [ ]:
# To check that the data has been properly loaded, call the method df.head()
df.head()

In [ ]:
# You get the number of rows and columns from the attribute df.shape
df.shape

In [ ]:
# We only want the TF name (column 1) and the TF binding site sequence (column 13)
# To keep only these columns, index the df using a list of column names you want (in the order you want)
col_names = [1,13]
df = df[col_names]
df.head()

In [ ]:
# Data frames allow users to give columns meaningful names.
# To rename the columns, set df.columns to a list of the desired names.
df.columns = ['tf','site']
df.head()

In [ ]:
# We see that some TF sites are listed as NaN. 
# Let's use the dropna() method to get rid of these rows.
df = df.dropna()
df.head()

In [ ]:
# Note that the last three modifications of df can be accomplished in one line 
df = pd.read_csv("data/binding_site_db.txt", sep='\t', comment='#',
                 header=None, usecols=[1,13], names=['tf','site']).dropna()
df.head()

In [ ]:
# Check out the pd.read_csv() documentation for a full list
pd.read_csv?

In [ ]:
# Dataframe columns are called 'Series' objects. 
# Essentially, they're numpy arrays with some extra structure.
col = df['tf']
col.head()

In [ ]:
# You can extract an element from a dataframe by using .loc[]
df.loc[3,'site']

Our goal is to generate sequence logos that represents the binding preferences of TFs in this database.  As a concrete example we'll use CRP, which has a well-characterized binding motif shown here:

<img src="https://github.com/bharris12/URP_2021_Programming_Course/blob/main/lecture_3/data/crp_information_logo.png?raw=1" alt="Drawing" style="width: 700px;"/>

## Subsetting dataframe


In [ ]:
# Choose a TF
tf = 'CRP'

# Flag which rows in the dataframe have the correct TF name
flags = (df['tf']==tf)
flags

In [ ]:
# Grab those rows. To be safe use copy() to make sure that, if we
# alter tf_df, df itself doesn't change
tf_df = df[flags].copy()
tf_df.head()

Selecting rows by a boolean vector, like `flags` is basically the only way I ever subset my rows. When you store rows as observations this is generally the case because you usually are **filtering** the rows based on a specific feature.

### All the ways to subset DataFrames

#### Rows
By boolean : `df[BOOLEAN_VECTOR]` 

By name : `df.loc[LIST_OF_ROW_NAMES, :]`

By Location : `df.iloc[LIST_OF_INTEGERS, :]`



With a function `df.query('COLUMMN==value')` (This one is quite advanced, see [this tutorial for more](https://www.sharpsightlabs.com/blog/pandas-query/))
#### Columns
By boolean : `df.loc[:,BOOLEAN_VECTOR]`

By name** : `df[LIST_OF_COLUMN_NAMES]`

By Location : `df.iloc[:,LIST_OF_INTEGERS]`

With a function : `df.filter(LOTS_OF_OPTIONS)` (This one is quite advanced, see [this tutorial for more](https://www.sharpsightlabs.com/blog/pandas-filter/))

In [ ]:
# Each DNA binding site should be capitalized.
# To do this, we reset 'site' column of tf_df

# Get list of capitalized sites and replace the 'site' column with this
capitalized_sites = [site.upper() for site in tf_df['site']]
tf_df['site'] = capitalized_sites
tf_df.head()

In [ ]:
# In order to derive a motif, all sites we analyze need to be the same length.
# It's good to check that this is actually the case.
# We therefore add a column to tf_df listing the length of each site

# Compute the length of each site and record this in a 'length' column
site_lengths = [len(site) for site in tf_df['site']]
tf_df['length'] = site_lengths
tf_df.head()

## Exercises, part 1

**E4.1.** Use the `unique()` method to find how many different length exists for the TF binding sites.

In [ ]:
# Answer here

**E4.2.** Apparently these sites have a bunch of different lengths. For a better understanding of this, write a `for` loop to determine how many sites there are of each length.

In [ ]:
# Answer here

## Part 2

In [ ]:
# Use value_counts() to see how many sites of each length there are. 
tf_df['length'].value_counts()

In [ ]:
# Use the mode() method to compute the most common binding site length
# Note that mode() returns a tuple, of which we need to manually extract the first element
length_mode = tf_df['length'].mode()[0]
length_mode

In [ ]:
# Flag rows having sites of the chosen length
flags = (tf_df['length']==length_mode)

# Only keep these rows
tf_df = tf_df[flags]
tf_df.head()

In [ ]:
# Now extract the 'site' column from tf_df
sites = tf_df['site']
sites.head()

In [ ]:
# Using logomaker.alignment_to_matrix function, compute the number of times each base occurs at each position
# Note that this returns a dataframe
counts_mat = logomaker.alignment_to_matrix(sites)
counts_mat.head()

In [ ]:
# This counts matrix can be visualized as a sequence logo
logomaker.Logo(counts_mat,center_values=True)

In [ ]:
#what we most commonly see in publications are actually "information" logos
info_df = logomaker.transform_matrix(counts_mat,from_type='counts',to_type='information')
logomaker.Logo(info_df)

## Exercises, part 2

**E4.3.** Fill out the function below so that the user can pass the name of any TF and get list of aligned sites back. Test that it works, e.g. on `tf='FNR'`, by getting a list of sites and making an information logo. Also test that it fails when it is supposed to.

In [ ]:
# Now let's turn this into a function 
def get_tf_sites(tf):
   
    # Load database
    df = pd.read_csv("data/binding_site_db.txt", sep='\t', comment='#',
                 header=None, usecols=[1,13], names=['tf','site']).dropna()
    
    # 
    # Fill in stuff here
    # 
    
    # Get sequence alignment and return it
    return tf_df['site']


In [ ]:
aligned_sites = get_tf_sites('FNR')
#create information logo from retrived aligned sites of FNR

## Part 3

Now that we've went through the basics of dataframe and dataframe subsetting, lets go through a few pandas function that is commonly used for dataframe manipulation. For easiness, we'll do the following parts in the palmer penguine toy dataset

In [ ]:
import seaborn as sns
df = load_penguins()
df = df.dropna()
df

#### pd.melt()

In [ ]:
#how we plot the most common bar plot
#You x axis is species, a categorical variable. 
#This means that we can use it to group/seperate the data by species 
sns.barplot(data=df, x='species', y='bill_length_mm')

What if I wanted to compare the distributions of different numerical variables. Say see how bill_length and bill_width compare (not within an observation)? 

To do this you need to make the data "tall" using the function pd.melt()

In [ ]:
bills_tall = pd.melt(df[['bill_length_mm', 'bill_depth_mm']])
bills_tall.head()

In [ ]:
bills_tall['variable'].unique()

In [ ]:
sns.barplot(data=bills_tall,x='variable',y='value')

Now, this is great, but you may notice that we have lost the information about which species each observation came from, when making the data tall, you can add another argument to melt that will bring with each value the species

In [ ]:
bills_tall_species = pd.melt(
    df[['bill_length_mm', 'bill_depth_mm', 'species']], id_vars='species', var_name='measurment',value_name='mm')
bills_tall_species.head()

In [ ]:
#Next lecture will be on data visulization! 
sns.barplot(data=bills_tall_species, x='measurment', y='mm', hue='species');

#### pd.concat()
It is extremely common for data to come in separate files. But we need to join the files together. To use pd.concat()

In [ ]:
sub_df_1 = df.iloc[:,0:4]
sub_df_1.head()

In [ ]:
sub_df_2 = df.iloc[:,4:]
sub_df_2.head()

In [ ]:
cat_df = pd.concat([sub_df_1, sub_df_2],axis=1)
cat_df.head()
#The axis=1 tells the function to stick the columns next to eachother, would hstack if we remove it

## Part 4

Finally, just a few commonly used statistic tests. Here we'll heavily use the package `scipy`. Its a Python library that have implemented a lot of existing statistic tests.

### Calculating Mann Whitney U test statistic:
 The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y.

In [ ]:
import scipy

adelie_flag = df['species'] == 'Adelie'
gentoo_flag = df['species'] == 'Gentoo'

adelie_bill_l = df[adelie_flag]['bill_length_mm']
gentoo_bill_l = df[gentoo_flag]['bill_length_mm']

scipy.stats.mannwhitneyu(adelie_bill_l, gentoo_bill_l, nan_policy ='omit')

### Calculating correlation coefficients
The Pearson correlation coefficient measures the linear relationship between two datasets.

In [ ]:
bill_l = df['bill_length_mm']
body_m = df['body_mass_g']
scipy.stats.pearsonr(bill_l,body_m)

### Calculating P values

We are not going into the complications around p-value. Here I'm just gonna simply show how to do the most simple t-test using python

In [ ]:
scipy.stats.ttest_ind(adelie_bill_l, gentoo_bill_l, equal_var=False)